# Seguridad y Privacidad de los Datos - 2024/25

Last edited: 2025/03/25

In [10]:
import os
import numpy as np
import pandas as pd
import onnxruntime as rt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from ydata_profiling import ProfileReport
import xgboost as xgb
from tqdm import tqdm
from torch import nn
import torch
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import random
import copy
import time

# 0. Train the model on the provided dataset

In [2]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(354, 64)
        self.fc2 = nn.Linear(64, 2)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)
    


In [3]:
df = pd.read_csv('dataset.csv')
df = df.drop(["timestamp",'unused_permissions',"sha256"], axis=1)

In [4]:
X_train,X_test,y_train,y_test = train_test_split(df.drop('label',axis=1),df['label'],test_size=0.15,random_state=42)

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = Model().to(device)
dataloader_train= DataLoader(CustomDataset(X_train.values, y_train.values), batch_size=64, shuffle=True)
dataloader_test= DataLoader(CustomDataset(X_test.values, y_test.values), batch_size=64, shuffle=False)

EPOCHS=10
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
for epoch in range(EPOCHS):
    model.train()
    for data, labels in tqdm(dataloader_train,desc="Training", total=len(dataloader_train)):
        optimizer.zero_grad()
        data, labels = data.to(device), labels.to(device)
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    correct=0
    total=0
    for data, labels in tqdm(dataloader_test,desc="Testing", total=len(dataloader_test)):
        data, labels = data.to(device), labels.to(device)
        outputs = model(data)
        loss = criterion(outputs, labels)
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    print(f"Epoch {epoch+1}/{EPOCHS}, Accuracy: {100 * correct / total:.2f}%")

Testing: 100%|██████████| 177/177 [00:00<00:00, 383.30it/s]


Epoch 1/10, Accuracy: 95.73%


Testing: 100%|██████████| 177/177 [00:00<00:00, 553.70it/s]


Epoch 2/10, Accuracy: 96.12%


Testing: 100%|██████████| 177/177 [00:00<00:00, 542.19it/s]


...

Epoch 8/10, Accuracy: 96.48%


Testing: 100%|██████████| 177/177 [00:00<00:00, 461.17it/s]


Epoch 9/10, Accuracy: 96.49%


Testing: 100%|██████████| 177/177 [00:00<00:00, 635.79it/s]

Epoch 10/10, Accuracy: 96.71%


In [4]:
num_round = 20
param = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'
dtrain = xgb.DMatrix(np.array(X_train), label=np.array(y_train))
dtest = xgb.DMatrix(np.array(X_test), label=np.array(y_test))
bst = xgb.train(param, dtrain, num_round,evals=[(dtest, 'test')])

[0]	test-auc:0.77060
[1]	test-auc:0.51361
[2]	test-auc:0.79185
[3]	test-auc:0.83762
[4]	test-auc:0.86815
[5]	test-auc:0.88486
[6]	test-auc:0.90663
[7]	test-auc:0.91467
[8]	test-auc:0.92638
[9]	test-auc:0.93567
[10]	test-auc:0.93776
[11]	test-auc:0.93818
[12]	test-auc:0.94127
[13]	test-auc:0.94256
[14]	test-auc:0.94401
[15]	test-auc:0.94171
[16]	test-auc:0.94205
[17]	test-auc:0.94187
[18]	test-auc:0.94029
[19]	test-auc:0.94379


In [5]:
ypred = bst.predict(dtest)
ypred = [1 if x > 0.5 else 0 for x in ypred]
print(classification_report(y_test, ypred))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97     10145
           1       0.82      0.66      0.73      1149

    accuracy                           0.95     11294
   macro avg       0.89      0.82      0.85     11294
weighted avg       0.95      0.95      0.95     11294



# 1. Perform at least one black-box attack and one white-box attack on the model. You may perform additional attacks as desired.

- Black box
- White box

In [36]:
no_attack=X_test[y_test == 0]
y_no_attack = y_test[y_test == 0]
dtest = xgb.DMatrix(np.array(no_attack), label=np.array(y_no_attack))
ypred = bst.predict(dtest)
ypred = [1 if x > 0.5 else 0 for x in ypred]
accuracy_score(y_no_attack, ypred)

0.9833415475603746

In [34]:
lista=[]

for i in tqdm(range(1,2000),desc='Iterating over activities count'):
    no_attacks_changed=no_attack.copy()
    no_attacks_changed['activities_count']=i
    dtest = xgb.DMatrix(np.array(no_attacks_changed), label=np.array(y_no_attack))
    ypred = bst.predict(dtest)
    ypred = [1 if x > 0.5 else 0 for x in ypred]
    lista.append(accuracy_score(y_no_attack, ypred))
    

Iterating over activities count: 100%|██████████| 1999/1999 [02:07<00:00, 15.72it/s]


In [79]:
n_individuals=100
n_generations=100
mutation_rate=0.1
population = np.random.randint(0, 2000, size=(n_individuals, 8))

class Genetic():
    def __init__(self, population_size: int,model,y_test,X_test):
        self.population = np.random.randint(0, 100, size=(population_size, 8)).tolist()
        self.fitnesses = []
        self.model=model
        self.y_test=y_test
        self.X_test=X_test

    def fitness(self,individual):
        test=self.X_test.copy()
        test[:,0]=individual[0]
        # test[:,1]=individual[1]
        # test[:,2]=individual[2]
        # test[:,3]=individual[3]
        test[:,4]=individual[4]
        test[:,5]=individual[5]
        test[:,6]=individual[6]
        # test[:,7]=individual[7]
        test = torch.tensor(test, dtype=torch.float32).to("cuda")
        predictions = self.model(test.to("cuda"))
        predictions = torch.argmax(predictions, dim=1)
        predictions = predictions.cpu().numpy()
        return accuracy_score(self.y_test, predictions)

    def sort_pop(self, reversed: bool) -> tuple[list[list], list]:
        """Sort population by fitness function. Return tuple with population list and fitness list"""

        fitness_list = [self.fitness(ind) for ind in self.population]
        lista = sorted(zip(self.population, fitness_list), key=lambda x: x[1], reverse=reversed)
        self.population = [x[0] for x in lista]
        self.fitnesses = [x[1] for x in lista]

    def select(self, T: int) -> list:
        """Return a copy of an individual by tournament selection. Population already ordered by fitness"""
        indices = random.choices(range(len(self.population)), k=T)
        best_index = min(indices)  # since population is ordered by fitness
        return copy.deepcopy(self.population[best_index])







    def mutate(self,individual: list[list], pmut: float) -> list:
        """Mutate an individual, swap elements. Return mutated individual"""

        def mutate_swap(individual: list[list], pmut: float) -> list[list]:
            if random.random() < pmut:
                index1, index2 = random.choices(range(len(individual)), k=2)
                individual[index1], individual[index2] = individual[index2], individual[index1]
            return individual

        def mutate_insert(individual: list[list], pmut: float) -> list[list]:
            if random.random() < pmut:
                index_pop, index_insert = random.choices(range(len(individual)), k=2)
                value = individual.pop(index_pop)
                individual.insert(index_insert, value)
            return individual

        def mutate_reverse(individual: list[list], pmut: float) -> list[list]: 
            if random.random() < pmut:
                index1, index2 = sorted(random.sample(range(len(individual)), 2))
                individual[index1:index2 + 1] = individual[index1:index2 + 1][::-1]
            return individual
        def mutate_random(individual: list[list], pmut: float) -> list[list]:
            if random.random()<pmut:
                for index in range(len(individual)):
                    if random.random() < pmut:
                        individual[index] =  np.random.randint(0,100)
            return individual
        
        mutations = [mutate_swap, mutate_insert, mutate_reverse,mutate_random]
        operator = random.choice(mutations)
        return operator(individual, pmut)

    def evolve(self,pmut=0.1, pcross=0.7, ngen=100, T=2, trace=50, reverse_sort=False, elitism=True) -> None:
        """Evolution procedure"""
        
        size = len(self.population)
        for i in range(ngen):
            new_pop = []
            self.sort_pop(reverse_sort)
            if elitism:
                new_pop.append(self.population[0])
                new_pop.append(self.population[1])
            while len(new_pop) != size:   
                individual1 = self.select(T)
                individual2 = self.select(T)
                # child1,child2 = self.crossover(individual1, individual2, pcross)
                mutated1 = self.mutate(individual1, pmut)
                mutated2 = self.mutate(individual2, pmut)
                new_pop.append(mutated1)
                new_pop.append(mutated2)
            
            if i % trace == 0 or i == ngen-1: # en la última gen se ordena
                self.sort_pop(reverse_sort)
                print(f"Num gen: {i}, Fitness óptimo: {self.fitnesses[0]}")
                print(f"Mejor individuo: {self.population[0]}")
            self.population = [*new_pop] # make a copy
        return self.population
    
start = time.time()
genetic_algorithm = Genetic(population_size=100, model=model, y_test=np.array(y_test), X_test=np.array(X_test)) 
final_population = genetic_algorithm.evolve(pmut=0.1, ngen=10, T=6, trace=1)
minutos, segundos = divmod(time.time()-start, 60)
print(f"*******Tiempo transcurrido: {int(minutos)} minutos y {segundos:.2f} segundos*******")

Num gen: 0, Fitness óptimo: 0.19886665486098815
Mejor individuo: [1, 84, 81, 24, 72, 55, 10, 15]
Num gen: 1, Fitness óptimo: 0.19886665486098815
Mejor individuo: [1, 84, 81, 24, 72, 55, 10, 15]
Num gen: 2, Fitness óptimo: 0.15388701965645477
Mejor individuo: [1, 84, 81, 24, 55, 72, 10, 15]
Num gen: 3, Fitness óptimo: 0.15388701965645477
Mejor individuo: [1, 84, 81, 24, 55, 72, 10, 15]
Num gen: 4, Fitness óptimo: 0.11785018593943687
Mejor individuo: [1, 84, 81, 24, 72, 95, 10, 15]
Num gen: 5, Fitness óptimo: 0.11785018593943687
Mejor individuo: [1, 84, 81, 24, 72, 95, 10, 15]
Num gen: 6, Fitness óptimo: 0.114662652735966
Mejor individuo: [1, 72, 24, 81, 84, 95, 10, 15]
Num gen: 7, Fitness óptimo: 0.114662652735966
Mejor individuo: [1, 72, 24, 81, 84, 95, 10, 15]
Num gen: 8, Fitness óptimo: 0.11430848238002479
Mejor individuo: [1, 84, 81, 24, 72, 95, 0, 15]
Num gen: 9, Fitness óptimo: 0.11430848238002479
Mejor individuo: [1, 84, 81, 24, 72, 95, 0, 15]
*******Tiempo transcurrido: 1 minuto

# 2. Prove that your model can be more robust to adversarial examples

Demonstrate that your model maintains high accuracy when subjected to adversarial examples. Provide empirical evidence of robustness by evaluating performance under adversarial attacks.

What Will Be Valued:
- Effective adversarial attack methods and proper defense mechanisms.
- Clear evaluation metrics comparing standard vs. adversarial performance.
- Well-documented methodology and reproducibility of results.

# 3. Try to attack this model

You will be given the anonymous model `android_permission_model.onnx`, previously trained on the provided dataset. 
Analyze the model’s weaknesses by testing different black-box attack strategies.

What Will Be Valued:
- Use of diverse adversarial attack techniques.
- Quantitative analysis of attack effectiveness with relevant metrics.
- Clear documentation of the attack process and reproducibility.

Below, you can find an example on model loading and inference. The model expects every column in the dataset except for `sha256`, `timestamp` and `unused_permissions`.
The model is loaded in an [onnx runtime](https://onnxruntime.ai/docs/api/python/api_summary.html). 

In [3]:


DATASET_ROUTE = './dataset.csv'

df = pd.read_csv(DATASET_ROUTE)
model_path = 'android_permission_model.onnx'

# removing unused columns
if 'timestamp' in df.columns:
    df = df.drop('timestamp', axis=1)
if 'unused_permissions' in df.columns:
    df = df.drop('unused_permissions', axis=1)
if 'sha256' in df.columns:
    df = df.drop('sha256', axis=1)

# features and target separation + train/test split
X = df.drop('label', axis=1)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

Example inference function using [sess.run()](https://onnxruntime.ai/docs/api/python/api_summary.html#onnxruntime.InferenceSession.run):

In [21]:
def onnx_inference(model_path, test_data):
    """
    Function to load and make predictions with an ONNX model
    """
    # open ONNX inference session
    sess = rt.InferenceSession(model_path)
    
    # input and output names
    input_name = sess.get_inputs()[0].name
    output_names = [output.name for output in sess.get_outputs()]
    
    # send inputs to the model and obtain predictions
    results = sess.run(output_names, {input_name: test_data.astype(np.float32)})

    return results[0]       # results[0] are discrete predicted labels, while results[1] are probabilities

In [22]:
# try and load the model
if not os.path.exists(model_path):
    print(f"Error: Model file '{model_path}' not found!")
    print(f"Current working directory: {os.getcwd()}")
    print(f"Files in current directory: {os.listdir('.')}")
else:
    # small sample to test if it works
    test_sample = X_test.iloc[:5].values
    onnx_predictions = onnx_inference(model_path, test_sample)
    
    if onnx_predictions is not None:
        print("\nONNX Model Predictions (sample):")
        print(onnx_predictions)
        
        # compare with actual labels
        actual_labels = y_test.iloc[:5].values
        print("\nActual labels:")
        print(actual_labels)
    
    # full test set now
    print("\nONNX Model Predictions (full test set):")
    full_predictions = onnx_inference(model_path, X_test.values)

    try:
        print(f"Accuracy: {accuracy_score(y_test, full_predictions):.4f}")
        print("\nClassification Report:")
        print(classification_report(y_test, full_predictions))
        print("\nConfusion Matrix:")
        print(confusion_matrix(y_test, full_predictions))
    except Exception as e:
        print(f"Error during ONNX prediction: {e}")


ONNX Model Predictions (sample):
[1 0 0 0 1]

Actual labels:
[1 0 0 0 0]

ONNX Model Predictions (full test set):
Accuracy: 0.9815

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     16943
           1       0.86      0.97      0.91      1880

    accuracy                           0.98     18823
   macro avg       0.93      0.98      0.95     18823
weighted avg       0.98      0.98      0.98     18823


Confusion Matrix:
[[16642   301]
 [   48  1832]]
